## 데이터 불러오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from google.colab import drive

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'sans-serif'

drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/f1_project/data/f1_race_data_2021_2025_merged.csv'

try:
    df = pd.read_csv(file_path)
    print(f"✅ 데이터 로드 성공")
    print(f"   - 총 데이터 크기: {df.shape} (행, 열)")
    print(f"   - 컬럼 목록: {df.columns.tolist()}")
    display(df.head())
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")

## 데이터 분석

### 1. 기초 통계 분석

In [ ]:
print("=== 데이터 기본 정보 ===")
df.info()

print("\n=== 기초 통계량 ===")
# 수치형 데이터의 평균, 표준편차, 4분위수 확인
display(df.describe().round(2))

print("\n=== 결측치 확인 ===")
print(df.isnull().sum())

- 총 100개 이상의 레이스 데이터가 수집되었으며, 결측치(Null)가 없어 별도의 대체 작업이 필요하지 않습니다.

- `Chaos_Score`의 평균은 약 4.38점이며, 표준편차가 커서 경기별로 혼란도의 차이가 큼을 알 수 있습니다.

### 2. 데이터 시각화

In [ ]:
numeric_cols = ['SC_Count', 'VSC_Count', 'Red_Flag_Count', 'Chaos_Score',
                'Lead_Changes', 'Gap_Std_Dev', 'Position_Gains_Total']

corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, vmin=-1, vmax=1)
plt.title('Correlation Matrix: F1 Race Excitement Features')
plt.show()

2.1 상관관계 분석

1.  **Chaos와 Action의 관계 (0.22)**:
    - `Chaos_Score`와 `Position_Gains_Total` 간에는 **약한 양의 상관관계**(0.22)가 나타났습니다.
    - **해석**: 세이프티 카(SC)나 레드 플래그가 발령되면 차량 간격이 좁혀져 변수가 발생하긴 하지만, 이것이 즉각적인 대량 추월로 이어지지는 않음을 시사합니다.
    - **원인 추정**: SC 상황에서의 랩 소모(추월 불가 구간 증가)와 모나코 등 추월이 어려운 서킷에서의 잦은 사고가 상관관계를 약화시킨 것으로 보입니다.

2.  **가설 검증**:
    - *"혼란스러울수록 추월이 많을 것이다"* 라는 초기 가설은 부분적으로만 유효하며, 추월에는 혼란도 외에도 서킷의 추월 용이성(Track Overtaking Difficulty) 등 다른 변수가 더 크게 작용할 수 있음을 발견했습니다.

In [ ]:
plt.figure(figsize=(12, 6))

sns.boxplot(x='Year', y='Chaos_Score', data=df, palette="Set3")
sns.stripplot(x='Year', y='Chaos_Score', data=df, color=".25", alpha=0.5)

plt.title('Yearly Distribution of Chaos Score (2021-2025)', fontsize=15)
plt.ylabel('Chaos Score (Weighted Index)')
plt.xlabel('Season')
plt.show()

2.2 연도별 혼란도(Chaos) 경향 분석
1.  **2021년**
    - 박스플롯의 IQR이 가장 크고 이상치(Outlier)가 많아 혼란도가 극심했던 해입니다.
    - 이는 루이스 해밀턴과 막스 베르스타펜의 치열한 경쟁 과정에서 잦은 충돌과 레드 플래그가 발생했음을 보여줍니다.

2.  **2024년**
    - 2024년은 맥라렌과 메르세데스의 컨스트럭터 경쟁이 매우 치열했음에도 불구하고, 혼란도 중앙값은 **2.0**으로 5년 중 가장 낮았습니다.
    - 이는 두 팀의 경쟁 과정 속 물리적 충돌이 별로 일어나지 않았다고 해석할 수 있습니다. 즉, '치열함'이 곧 '혼란(Chaos)'으로 이어지지는 않는다고 볼 수 있습니다.

3.  2025년의 변화:
    - 2025년 데이터는 중앙값이 다시 **3.0**으로 상승하며, 전년도에 비해 경기 내 변수가 다소 증가하는 경향을 보였습니다.

**[결론]**
데이터 분석 결과, 단순히 시즌의 치열함이 혼란도와 직결되지는 않으며, 2023년 호주 GP처럼 단일 경기의 특수성이 전체 데이터의 이상치를 주도할 수 있음을 확인했습니다.

In [ ]:
track_stats = df.groupby('GrandPrix')[['Position_Gains_Total', 'Lead_Changes']].mean().reset_index()

top_tracks = track_stats.sort_values('Position_Gains_Total', ascending=False).head(5)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_tracks, y='GrandPrix', x='Position_Gains_Total', palette='magma')
plt.title('Top 5 Circuits with Most Action (Position Gains)', fontsize=15)
plt.xlabel('Average Position Gains')
plt.ylabel('Grand Prix')
plt.show()

2-3. 서킷별 평균 순위 변동 분석 (Position Gains)

1.  **러시아 그랑프리 (Russian GP)**
    - **분석**: 2021년 단 한 번의 데이터만 포함되어 있으며, 당시 경기 막판 우천으로 인한 대혼란이 기록적인 순위 변동을 만들어내 평균값을 높였습니다. (현재는 캘린더 제외됨)

2.  **라스베이거스 그랑프리 (Las Vegas GP)**
    - **분석**: 긴 직선 구간(The Strip)을 활용한 추월 기회가 많아, 시가지 서킷임에도 불구하고 매우 높은 수치를 기록했습니다.
    - 표본은 3개로 적지만, 사고가 없었던 2024년에도 높은 추월 횟수를 기록한 점을 볼 때, 긴 직선 구간을 활용한 추월 용이성은 서킷의 고유한 특징으로 판단할 수 있습니다.

**[인사이트 & 한계점]**
- 단순 평균값 비교 시, 개최 횟수가 적은 서킷(예: 러시아)의 특이 케이스가 전체 순위를 왜곡할 수 있음을 확인했습니다. 따라서 향후 모델링 시에는 3회 이상 개최된 서킷만 필터링하는 방식의 전처리가 유효할 것으로 판단하였습니다.

- 또한, 본 분석에서 사용한 Position_Gains는 출발과 도착 순위만을 비교한 지표입니다.

- 따라서 레이스 도중 서로 순위를 뺏고 뺏기는 치열한 배틀(Battle)이나, 피트 스톱으로 인한 일시적 순위 변화 등 '실제 추월(Overtakes)' 횟수보다는 수치가 낮게 측정되는 경향이 있습니다.

- 하지만 계산이 빠르고, "결과적으로 얼마나 많은 변동이 있었는가"를 보여주는 **이변의 척도**로서는 여전히 유효하다고 판단하였습니다.